In [8]:
import os
import re
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Input widgets
Place = widgets.Text(
    value='',
    placeholder='Ghana,CIV,BF,Nigeria,Senegal,Cameroon,Mali',
    description='Country:',
    disabled=False
)

Path = widgets.Text(
    value='',
    placeholder='/Users/NAME/Documents/Data.csv',
    description='Path of file:',
    disabled=False
) 

button_submit = widgets.Button(description="Submit")


def fix_data(data):
    # CORRECT NUMBER FORMAT
    def correct_number(data):
        data['Numbers'] = data['Numbers'].astype(str)

        def remove_extras(sphy):
            return re.sub(r'[^0-9/.]', '', sphy)
        
        
        data["Numbers"] = data["Numbers"].apply(remove_extras)
        data["Numbers"] = data["Numbers"].astype(str).str.split('.').str[0]
        data["Numbers"] = data["Numbers"].str.split('/').str[0]

        # Define Country
        data["COUNTRY"] = Place.value.lower()  # Convert to lowercase for case-insensitive comparison
        # Add country-specific prefixes
        country_prefixes = {
            "ghana": "233",
            "nigeria": "234",
            "civ": "225",
            "mali": "223",
            "senegal": "223",
            "bf": "226",
            "cameroon": "237"
        }

        # Use the get method to handle the case when the country is not in the dictionary
        
        if not data.empty and len(data) > 0:
    
            if 0 in data.index:
                prefix = country_prefixes.get(data['COUNTRY'].iloc[0], '')
            else:
                prefix = ""
        else:
            prefix = ""

        mask = (~data['Numbers'].str.startswith(prefix)) & (data['Numbers'].str.len() <= 9) & (data['Numbers'].str.startswith(('2', '5')))
        data.loc[mask, 'Numbers'] = prefix + data.loc[mask, 'Numbers']

        return data

    # CORRECT NAME FORMAT - Remove emojis
    def remove_emojis(text):
        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002500-\U00002BEF"  # Chinese characters
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   u"\U0001f926-\U0001f937"
                                   u"\U00010000-\U0010ffff"
                                   u"\u2640-\u2642"
                                   u"\u2600-\u2B55"
                                   u"\u200d"
                                   u"\u23cf"
                                   u"\u23e9"
                                   u"\u231a"
                                   u"\ufe0f"  # dingbats
                                   u"\u3030"
                                   "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', text)

    # Drop rows with NaN
    data.dropna(inplace=True)

    # Apply remove_emojis to Name column
    data['Name'] = data['Name'].apply(remove_emojis)

    
    
    
    def change_emails(text):
    
        def extract_name(email):
            match = re.match(r'(.*?)@', email)
            if match:
                return match.group(1)
            return email
        
        
            return text.apply(extract_name)  

    data['Name'] = change_emails(data['Name'])     



    def change_text(text):
        preclean= re.sub(r'[._]',' ',text)
        clean = re.sub(r'[^A-Za-z ]','',preclean)
        if len(clean) < 3:
            return 'Dear'
        else:
            return clean

    data['Name'] = data['Name'].apply(change_text)

    return correct_number(data)


def on_button_submit_click(b):
    country = Place.value
    file_path = Path.value

    try:
        data = pd.read_excel(file_path) if file_path.lower().endswith(('.xlsx', '.xls')) else pd.read_csv(file_path)

        corrected_data = fix_data(data)
        corrected_data = corrected_data[corrected_data["Response3"].str.lower().isin(['yes', 'oui'])]
        corrected_data = corrected_data[['Numbers', 'Name']]
        

        display(corrected_data.head())

        save_path = os.path.join(os.path.dirname(os.path.abspath('__file__')), f'Corrected_{country}.csv')

        button_download = widgets.Button(description="Download Corrected Data")

        def on_button_download_click(b):
            corrected_data.to_csv(save_path, index=False)
            print(f"Data saved to: {save_path}")

        button_download.on_click(on_button_download_click)
        display(button_download)

    except Exception as e:
        print(f"Error reading the file: {e}")

# Set the event handler for the button click
button_submit.on_click(on_button_submit_click)


# Display widgets
display(Place, Path, button_submit)


Text(value='', description='Country:', placeholder='Ghana,CIV,BF,Nigeria,Senegal,Cameroon,Mali')

Text(value='', description='Path of file:', placeholder='/Users/NAME/Documents/Data.csv')

Button(description='Submit', style=ButtonStyle())

Error reading the file: single positional indexer is out-of-bounds
